# Parsing Logs from a MUD for Description Information

In [1]:
import os, sys
import re
import json
from triggers import *
from pathlib import Path

## 1.) Collect and load Logs from text source

In [40]:
data_folder = Path("data/mudlogs/")
idx = 0
docs = {}
document = []
for file in data_folder.iterdir():
    if str(file).endswith("test.log"):
        file_to_open = file
        o = open(file_to_open, 'r')
        documentName = idx
        idx += 1
        document = list(o)
        docs[documentName] = document
print(docs[0])

In [3]:
for line in document:
    ansi_rem = re.compile(r'\x1B\[[0-?]*[ -/]*[@-~](\\?\;?\d+)*m')
    
    print(ansi_rem.sub('', line))

{"mud": "sloth", "line": "\u001b[0\\;37mYou recall at least one encounter.  Counting beyond that is hard with all this equipment on your toes.\u001b[0\\;37\\;40m", "tag": "mudline", "user": "final", "timestamp": 1529199267.401862}

{"mud": "sloth", "line": "\u001b", "tag": "mudline", "user": "final", "timestamp": 1529199267.405662}

{"mud": "sloth", "line": "\u001b[0\\;37m<\u001b[0\\;32m388\u001b[0\\;37mhp \u001b[0\\;32m274\u001b[0\\;37mma \u001b[0\\;33m89\u001b[0\\;37mmv -%>\u001b[0\\;37m", "tag": "mudline", "user": "final", "timestamp": 1529199267.409178}

{"mud": "sloth", "line": "", "tag": "mudline", "user": "final", "timestamp": 1529199267.41301}

{"mud": "sloth", "line": "order undead kill spider\r\n", "tag": "playerline", "user": "final", "timestamp": 1529199268.698225}

{"mud": "sloth", "line": "kill spider\r\n", "tag": "playerline", "user": "final", "timestamp": 1529199268.703387}

{"mud": "sloth", "line": "\u001b[0\\;33mA spectre claws a creepy spider extremely hard.\u001b[0\

In [37]:
for line in document:
    ansi_rem = re.split(r'(37m)|(33m)', line, 100)
    stripFront = ansi_rem
    l = len(ansi_rem)
    #print(stripFront[l-1])
    ansi_rem2 = re.split(r'(\\)', stripFront[l-1], 100)
    stripBack = ansi_rem2
    print(stripBack[0])

You recall at least one encounter.  Counting beyond that is hard with all this equipment on your toes.
{"mud": "sloth", "line": "
", "tag": "mudline", "user": "final", "timestamp": 1529199267.409178}

{"mud": "sloth", "line": "", "tag": "mudline", "user": "final", "timestamp": 1529199267.41301}

{"mud": "sloth", "line": "order undead kill spider
{"mud": "sloth", "line": "kill spider
A spectre claws a creepy spider extremely hard.
The field mouse looks around for signs of danger.
A ghoul of a drowsy black bear pounds a creepy spider very hard.
{"mud": "sloth", "line": "
", "tag": "mudline", "user": "final", "timestamp": 1529199269.134987}

{"mud": "sloth", "line": "", "tag": "mudline", "user": "final", "timestamp": 1529199269.138914}

{"mud": "sloth", "line": "cast 'fireball'
A spectre claws a creepy spider extremely hard.
A spectre claws a creepy spider extremely hard.
A ghoul of a drowsy black bear pounds a creepy spider very hard.
A ghoul of a drowsy black bear misses a creepy spider

In [5]:
import collections
import re

Token = collections.namedtuple('Token', ['type', 'value', 'line', 'column'])

def tokenize(code):
    keywords = {'IF', 'THEN', 'ENDIF', 'FOR', 'NEXT', 'GOSUB', 'RETURN'}
    token_specification = [
        ('NUMBER',   r'\d+(\.\d*)?'),  # Integer or decimal number
        ('ASSIGN',   r':='),           # Assignment operator
        ('END',      r';'),            # Statement terminator
        ('ID',       r'[A-Za-z]+'),    # Identifiers
        ('OP',       r'[+\-*/]'),      # Arithmetic operators
        ('NEWLINE',  r'\n'),           # Line endings
        ('SKIP',     r'[ \t]+'),       # Skip over spaces and tabs
        ('MISMATCH', r'.'),            # Any other character
    ]
    tok_regex = '|'.join('(?P<%s>%s)' % pair for pair in token_specification)
    line_num = 1
    line_start = 0
    for mo in re.finditer(tok_regex, code):
        kind = mo.lastgroup
        value = mo.group()
        column = mo.start() - line_start
        if kind == 'NUMBER':
            value = float(value) if '.' in value else int(value)
        elif kind == 'ID' and value in keywords:
            kind = value
        elif kind == 'NEWLINE':
            line_start = mo.end()
            line_num += 1
            continue
        elif kind == 'SKIP':
            continue
        elif kind == 'MISMATCH':
            raise RuntimeError(f'{value!r} unexpected on line {line_num}')
        yield Token(kind, value, line_num, column)

statements = '''{"mud": "sloth", "line": "\u001b[0\\;37mYou recall at least one encounter.  Counting beyond that is hard with all this equipment on your toes.\u001b[0\\;37\\;40m", "tag": "mudline", "user": "final", "timestamp": 1529199267.401862}'''

for token in tokenize(statements):
    print(token)

RuntimeError: '{' unexpected on line 1

## 3.) Keras Encoder or LSTM-DQN from "Playing..."

## 4.) Sentiment Analysis (optional)

## 5.) Decoder

## 6.) Autoencoder with two outputs: reconstructed input and predicted sentiment